# Ecole du Technologie Supreme Programming Assignment

Study the issues that are related to the rails project: https://github.com/rails/rails/issues. Each of these issues has a specific category that is indicated by its label (e.g., actionpack, rails-ujs, regression, …). On top of that, each issue has a set of possible metrics that we can collect, such as the number of comments that are associated with an issue.

**Write a script that collects the last 500 issues and investigate the following questions:**


1. How do the number of issues evolve across time?

2. Are there any periods in which we get more issues?

3. Is there anyone who reports more issues than others?

4. What is the most popular category (label)? 

5. Suggest a question that you came up with yourself (please explain the question in your documentation).

## Solution

### Step 1: Data Collection from Github

In [1]:
# import the required libraries
import requests
import json
import pandas as pd
import config
import ast

In [2]:
# install plotly for graph
# pip install plotly==5.7.0
# import plotly library
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [3]:
# instantiate a request session
issue_session = requests.Session()
# provide github rest api credential from the config
issue_session.auth = (config.GITHUB_USERNAME, config.GITHUB_TOKEN)

In [4]:
# setup a function to iterate through the pages of all issues in the repo
def load_github_repo_issues(repo, owner, github_api):
    # create empty placeholder for issues
    issues = []
    # since github paga pagination is defaulted to 30
    # we loop through all the pages
    next = True
    pg_int = 1
    while next == True:
        # construct a query complete github rest api query 
        url = github_api + '/repos/{}/{}/issues?page={}&per_page=100&status=all'.format(owner, repo, pg_int)
        issue_pg = issue_session.get(url = url)
        issue_pg_list = [dict(issue_item) for issue_item in issue_pg.json() ]
        issues = issues + issue_pg_list
        # check for pagination
        if 'Link' in issue_pg.headers:
            if 'rel="next"' not in issue_pg.headers['Link']:
                next = False
        #increment page counter
        pg_int +=1
        
    # return normalized issues    
    return pd.json_normalize(issues)

In [8]:
# load the last 500 records
dataset = load_github_repo_issues('rails', 'rails', config.GITHUB_REST_API)[-500:]

In [9]:
# save the dataset
dataset.to_csv('dataset/issues.csv')

In [10]:
# view the dataset information
dataset.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 111 to 610
Data columns (total 110 columns):
 #    Column                                 Dtype  
---   ------                                 -----  
 0    url                                    object 
 1    repository_url                         object 
 2    labels_url                             object 
 3    comments_url                           object 
 4    events_url                             object 
 5    html_url                               object 
 6    id                                     int64  
 7    node_id                                object 
 8    number                                 int64  
 9    title                                  object 
 10   labels                                 object 
 11   state                                  object 
 12   locked                                 bool   
 13   assignee                               float64
 14   assignees                             

### Step 2: Data Analysis

In [5]:
# import the saved dataset
issues = pd.read_csv('dataset/issues.csv', parse_dates=True)

In [6]:
# drop unneccessary columns
issues = issues[['id','title','user.id','state','locked','assignee.login','created_at','updated_at','author_association','user.login','reactions.total_count','comments','labels']]

In [7]:
# check the top ten dataset
issues.head(10)

,id,title,user.id,state,locked,assignee.login,created_at,updated_at,author_association,user.login,reactions.total_count,comments,labels
0,1161580864,Refer to number_field_tag in range_field docum...,1102934,open,False,NaN,2022-03-07T15:37:34Z,2022-03-07T15:39:57Z,CONTRIBUTOR,dmarcoux,1,0,"[{'id': 150377, 'node_id': 'MDU6TGFiZWwxNTAzNz..."
1,1160790017,Restart Action Cable when Redis disconnects,300588,open,False,NaN,2022-03-07T01:23:51Z,2022-03-22T09:11:33Z,NONE,engwan,1,1,"[{'id': 300028327, 'node_id': 'MDU6TGFiZWwzMDA..."
2,1160692008,Avoid double type cast when serializing attrib...,771968,open,False,NaN,2022-03-06T19:15:50Z,2022-03-15T20:29:23Z,MEMBER,jonathanhefner,0,6,"[{'id': 107190, 'node_id': 'MDU6TGFiZWwxMDcxOT..."
3,1160614719,System tests screenshots - include path in tes...,6763624,open,False,NaN,2022-03-06T13:44:40Z,2022-03-08T06:12:02Z,NONE,davidwessman,0,2,[]
4,1159903813,Always preload if using proc with multifetch c...,509837,open,False,NaN,2022-03-04T17:55:29Z,2022-03-08T01:12:46Z,MEMBER,ghiculescu,0,0,"[{'id': 3666649, 'node_id': 'MDU6TGFiZWwzNjY2N..."
5,1159838401,Add :renderable to the list of rendering keys ...,922012,open,False,NaN,2022-03-04T16:42:49Z,2022-03-16T23:44:01Z,NONE,voraciousdev,0,0,"[{'id': 3666649, 'node_id': 'MDU6TGFiZWwzNjY2N..."
6,1158163113,Add Module#eager_load! to trigger the loading ...,19192189,open,False,NaN,2022-03-03T09:20:49Z,2022-03-03T09:20:53Z,CONTRIBUTOR,casperisfine,1,0,"[{'id': 107194, 'node_id': 'MDU6TGFiZWwxMDcxOT..."
7,1157622253,Make `timestamptz` a time zone aware type for ...,509837,open,False,NaN,2022-03-02T19:53:12Z,2022-03-04T17:58:43Z,MEMBER,ghiculescu,3,1,"[{'id': 107191, 'node_id': 'MDU6TGFiZWwxMDcxOT..."
8,1157444988,Document behaviour when `:touch` is used with ...,47113995,open,False,NaN,2022-03-02T16:50:09Z,2022-03-02T16:50:12Z,NONE,mansakondo,0,0,"[{'id': 107191, 'node_id': 'MDU6TGFiZWwxMDcxOT..."
9,1157057672,fix hash from_xml missing tag error,4384971,open,False,NaN,2022-03-02T11:23:02Z,2022-03-18T03:47:59Z,NONE,hxx,0,3,"[{'id': 107194, 'node_id': 'MDU6TGFiZWwxMDcxOT..."


**We are going to generate some time related data from the issue created_at column using the Panda dt accessor, these are need to answer some questions**

In [11]:
issues['date'] = pd.to_datetime(issues['created_at'], utc=True)

In [12]:
issues['issue_date'] = issues['date'].dt.date

In [13]:
issues['issue_hour'] = issues['date'].dt.hour

In [14]:
issues['issue_month'] = issues['date'].dt.month

In [15]:
issues['issue_year'] = issues['date'].dt.year

In [16]:
issues.head()

,id,title,user.id,state,locked,assignee.login,created_at,updated_at,author_association,user.login,reactions.total_count,comments,labels,date,issue_date,issue_hour,issue_month,issue_year
0,1161580864,Refer to number_field_tag in range_field docum...,1102934,open,False,NaN,2022-03-07T15:37:34Z,2022-03-07T15:39:57Z,CONTRIBUTOR,dmarcoux,1,0,"[{'id': 150377, 'node_id': 'MDU6TGFiZWwxNTAzNz...",2022-03-07 15:37:34+00:00,2022-03-07,15,3,2022
1,1160790017,Restart Action Cable when Redis disconnects,300588,open,False,NaN,2022-03-07T01:23:51Z,2022-03-22T09:11:33Z,NONE,engwan,1,1,"[{'id': 300028327, 'node_id': 'MDU6TGFiZWwzMDA...",2022-03-07 01:23:51+00:00,2022-03-07,1,3,2022
2,1160692008,Avoid double type cast when serializing attrib...,771968,open,False,NaN,2022-03-06T19:15:50Z,2022-03-15T20:29:23Z,MEMBER,jonathanhefner,0,6,"[{'id': 107190, 'node_id': 'MDU6TGFiZWwxMDcxOT...",2022-03-06 19:15:50+00:00,2022-03-06,19,3,2022
3,1160614719,System tests screenshots - include path in tes...,6763624,open,False,NaN,2022-03-06T13:44:40Z,2022-03-08T06:12:02Z,NONE,davidwessman,0,2,[],2022-03-06 13:44:40+00:00,2022-03-06,13,3,2022
4,1159903813,Always preload if using proc with multifetch c...,509837,open,False,NaN,2022-03-04T17:55:29Z,2022-03-08T01:12:46Z,MEMBER,ghiculescu,0,0,"[{'id': 3666649, 'node_id': 'MDU6TGFiZWwzNjY2N...",2022-03-04 17:55:29+00:00,2022-03-04,17,3,2022


In [17]:
# group the issue created per hour
issues_by_hour = issues.groupby('issue_hour')[['id']].count()
issues_by_hour = issues_by_hour.rename(columns = {'id': 'issue_count'})

In [18]:
# plot a bar chart for hourly issues
fig = go.Figure([go.Bar(
    x=issues_by_hour.index, 
    y=issues_by_hour.issue_count, 
    text=issues_by_hour.issue_count, 
    textposition='auto')])
fig.update_layout(
    title = 'Issues by Hour', 
    xaxis_title = 'Hour', 
    yaxis_title = 'Issue Count', 
    xaxis_tickmode = 'linear')
fig.show()

In [19]:
# Let investigate further the issue activities of the repo over the years
#group the issue created per date
issues_by_day = issues.groupby('issue_date')[['id']].count()
issues_by_day = issues_by_day.rename(columns = {'id': 'issue_count'})
# Using time-series graph for issues over the year
fig = go.Figure([go.Scatter(
    x=issues_by_day.index, 
    y=issues_by_day.issue_count, 
    text=issues_by_day.issue_count, 
    fill='tozeroy')])
fig.update_layout(
    title = 'Issues by Date', 
    xaxis_title = 'Date', 
    yaxis_title = 'Issue Count')
fig.show()

In [27]:
# To reinforce that more issues are generated in 2021 than 2022. 
# At the time of the analysis, 2022 last issue was in Mar-22
# we compute daily average of issues
yearly_issues = issues.groupby('issue_year')[['id']].count()
yearly_issues = yearly_issues.rename(columns = {'id': 'issues_count'})
yearly_issues['daily_avg'] = yearly_issues['issues_count']/365
fig = go.Figure([go.Scatter(
    x=yearly_issues.index, 
    y=yearly_issues.daily_avg, 
    text=yearly_issues.daily_avg
)])
fig.update_layout(
    title = 'Average Daily Issues Reported by Year', 
    xaxis_title = 'Year', 
    yaxis_title = 'Average Number of Issues',
    xaxis_tickmode = 'linear')
fig.show()

In [21]:
# Let us determine the cumulative month that have the highest issues
monthly_issues = issues.groupby('issue_month')[['id']].count()
monthly_issues = monthly_issues.rename(columns = {'id': 'issues_count'})

fig = go.Figure([go.Scatter(
    x=monthly_issues.index, 
    y=monthly_issues.issues_count, 
    text=monthly_issues.issues_count
)])
fig.update_layout(
    title = 'Monthly Issues', 
    xaxis_title = 'Month', 
    yaxis_title = ' Number of Issues',
    xaxis_tickmode = 'linear')
fig.show()

In [22]:
# let determine the top user rank of issue reported on the repo
issues_by_user = issues.groupby('user.login')[['id']].count()
issues_by_user = issues_by_user.rename(columns = {'id': 'issue_count'})
issues_by_user = issues_by_user.sort_values(by='issue_count', ascending=False)
top_users = issues_by_user.head(40)
fig = go.Figure([go.Bar(
    x=top_users.index, 
    y=top_users.issue_count)])
fig.update_layout(
    title = 'Top Issue Reporters', 
    xaxis_title = 'Users', 
    yaxis_title = 'Issue Count', 
    xaxis_tickmode = 'linear',
    xaxis_tickangle=-40)
fig.show()

In [23]:
# create a list from a dictionary
labels_list = [ast.literal_eval(i) for i in issues['labels'].tolist() if i != '[]']

In [24]:
labels = [j for i in labels_list for j in i]

In [25]:
# convert labels to dataframe
labels_df = pd.DataFrame(labels)

In [26]:
labels_df 

,id,node_id,url,name,color,default,description
0,150377,MDU6TGFiZWwxNTAzNzc=,https://api.github.com/repos/rails/rails/label...,docs,02d7e1,False,None
1,3666649,MDU6TGFiZWwzNjY2NjQ5,https://api.github.com/repos/rails/rails/label...,actionview,d7e102,False,None
2,300028327,MDU6TGFiZWwzMDAwMjgzMjc=,https://api.github.com/repos/rails/rails/label...,actioncable,bfdadc,False,None
3,107190,MDU6TGFiZWwxMDcxOTA=,https://api.github.com/repos/rails/rails/label...,activemodel,00E5FF,False,None
4,107191,MDU6TGFiZWwxMDcxOTE=,https://api.github.com/repos/rails/rails/label...,activerecord,0b02e1,False,None
...,...,...,...,...,...,...,...
896,149514554,MDU6TGFiZWwxNDk1MTQ1NTQ=,https://api.github.com/repos/rails/rails/label...,pinned,f7c6c7,False,None
897,3666649,MDU6TGFiZWwzNjY2NjQ5,https://api.github.com/repos/rails/rails/label...,actionview,d7e102,False,None
898,149514554,MDU6TGFiZWwxNDk1MTQ1NTQ=,https://api.github.com/repos/rails/rails/label...,pinned,f7c6c7,False,None
899,107189,MDU6TGFiZWwxMDcxODk=,https://api.github.com/repos/rails/rails/label...,actionpack,FFF700,False,None


In [28]:
labels_by_name = labels_df.groupby('name')[['id']].count()
labels_by_name = labels_by_name.rename(columns = {'id': 'label_count'})
labels_by_name = labels_by_name.sort_values(by=['label_count'], ascending=False)

In [29]:
labels_by_name

,label_count
name,
activerecord,222
attached PR,108
With reproduction steps,93
pinned,68
actionpack,67
actionview,41
activestorage,36
activesupport,35
railties,33


In [30]:
fig = go.Figure([go.Bar(
    x=labels_by_name.index, 
    y=labels_by_name.label_count)])
fig.update_layout(
    title = 'Most popular Label Category', 
    xaxis_title = 'Labels', 
    yaxis_title = 'Label Count', 
    xaxis_tickmode = 'linear',
    xaxis_tickangle=-40)
fig.show()

In [31]:
# top commenters
comment_by_users = issues.groupby('user.login')[['comments']].sum()
comment_by_users = comment_by_users.rename(columns = {'comments': 'total_comment'})
comment_by_users = comment_by_users.sort_values(by=['total_comment'], ascending=False)

In [32]:
top_commenters = comment_by_users.head(40)
fig = go.Figure([go.Bar(
    x=top_commenters.index, 
    y=top_commenters.total_comment)])
fig.update_layout(
    title = 'Top Commenters on Issue', 
    xaxis_title = 'Users', 
    yaxis_title = 'Total Comment Count', 
    xaxis_tickmode = 'linear',
    xaxis_tickangle=-40)
fig.show()